# Load Data

> Functions that load the data for the map.   

**Contents**

`Solomon Geospatial Data`
- `SolomonGeo`: A class that cleans the solomon islandas census and geography data
- `SolomonGeo.read_test`: Loads and transforms the test data
- `SolomonGeo.get_geojson`: Returns the geo_df as a geojson datatset

In [ ]:
#| default_exp load_data

In [ ]:
#| export
from nbdev.showdoc import *
import geopandas as gpd
import pandas as pd
from git import Repo
import json
from fastcore import *
from fastcore.basics import patch
from fastcore.test import *
import sys
import topojson as tp
import pickle
from urllib.request import urlopen
import boto3
from dotenv import load_dotenv
from dash import dcc
import os
import copy

load_dotenv()

True

In [ ]:
#| hide
repo = Repo('.', search_parent_directories=True)
fp = str(repo.working_tree_dir) + "/testData/"
const_df = pd.read_csv(fp + 'sol_census_' + 'all' + '_' + '2009_v2.csv')
# Check that the files exist using fastcore (both census and geo)

## Solomon Geospatial Data
> Load the geography and census data
### Geography Data
Solomons islands geography data is organised at the levels
 - adm0 - The country as as whole, Solomon Islands
 - adm1 - Also referred to as the province e.g. Honiara, Malaita
 - adm2 - The Consituency e.g. Central Honiara
 - adm3 - Ward, the smallest geography I am reporting. E.g. Cruz

### Census Data
 Solomon islands census data has been used from the 2009 and 2019 census. For the respective census:
2009
 - We have the total population in for each of the administration regions
2019
 - There is only data available down to the province level

### Test that the .env variables exist

In [ ]:
#| hide
ACCESS_KEY = os.getenv("ACCESS_KEY")
SECRET_ACCCESS_KEY = os.getenv("SECRET_ACCESS_KEY")
REGION_NAME = os.getenv("REGION_NAME")
if len(ACCESS_KEY) == 0:
    # If not in .env, then use environment variables
    ACCESS_KEY = os.environ["ACCESS_KEY"]
    SECRET_ACCCESS_KEY = os.environ["SECRET_ACCESS_KEY"]
    REGION_NAME = os.environ["REGION_NAME"]

In [ ]:
# Test that the environment variables can be loaded
test(ACCESS_KEY, None, nequals)
test(SECRET_ACCCESS_KEY, None, nequals)
test(REGION_NAME, None, nequals)
test_eq(REGION_NAME, 'ap-southeast-2')

## Function for connecting to AWS S3 client

In [ ]:
#| export
def s3_client()-> boto3.client:
    '''Return a connection to teh AWS s3 client'''
    ACCESS_KEY = os.getenv("ACCESS_KEY")
    SECRET_ACCCESS_KEY = os.getenv("SECRET_ACCESS_KEY")
    REGION_NAME = os.getenv("REGION_NAME")
    if len(ACCESS_KEY) == 0:
        # If not in .env, then use environment variables
        ACCESS_KEY = os.environ["ACCESS_KEY"]
        SECRET_ACCCESS_KEY = os.environ["SECRET_ACCESS_KEY"]
        REGION_NAME = os.environ["REGION_NAME"]
    session = boto3.Session(region_name='ap-southeast-2')
    # Creating the low level functional client
    return session.client(
        's3',
        endpoint_url='https://s3.ap-southeast-2.amazonaws.com',
        aws_access_key_id = ACCESS_KEY,
        aws_secret_access_key = SECRET_ACCCESS_KEY,
        region_name = REGION_NAME,
    )

In [ ]:
#| hide
show_doc(s3_client)

---

### s3_client

>      s3_client ()

Return a connection to teh AWS s3 client

In [ ]:
#| export
class SolomonGeo:
    # TODO work out how to format the attributes
    # Look at nbdev docs maybe?
    # TODO change all data to int?
    # TODO - should I make this a dataclass for the auto functionaliy? potentially should try it out
    '''
    Load the solomon islands geography data 
    Attributes:
        cen_df    Geopandas dataframe containing geographies and census data
        geo_levels    A list of the types of available aggregations
        census_vars    A dictionary of census variables in the dataset 
        data_type   Specifies whether the variable is a percentage or number
        locations A dictionary of locations accessed by the geography level
    '''
    def __init__(self, 
                cen_df:pd.DataFrame, # A dataset containing the census data,
                pop_df:pd.DataFrame, # A dataset contain the population projection data
                geos:gpd.GeoDataFrame, # A geodataframe containing geographies of data
    ):
        self.census = cen_df
        self.population = pop_df
        self.geo = geos

        # variable that tracks the types of aggregations
        self.geo_levels = cen_df.loc[:, ('core', 'agg')].unique()

        # Save a list of census variables, ignoring the core variables
        # Use a dictionary that maps the upper level column names to lower level ones
        var_df = cen_df.drop(columns = "core", level=0)
        vars = {}
        for col in var_df.columns:
            if col[0] not in vars:
                vars[col[0]] = [col[1]]
            else:
                vars[col[0]].append(col[1])
        self.census_vars = vars

        # TODO should captialise first letter
        self.data_type = cen_df.loc[:, ('core', 'type')].unique()

        # save a list of locations as a dictionary access by geography level
        locations = {}
        for geo in self.geo_levels:
            locations[geo] = cen_df.loc[cen_df['core']['agg'] == geo, ('core', 'location')].unique()
        self.locations = locations
        # TODO: need a list of column sub headings: get from column name split by `:`

        self.type_default = 'Total'


    @classmethod
    def read_test(cls,
                 ): # A solmon geo class TODO work out how to return self here... (can't?)
        '''
        Contsructor that initialises the object from files using the local testing data
        '''

        repo = Repo('.', search_parent_directories=True)
        pw = str(repo.working_tree_dir) + "/testData/"
        df = pd.read_csv(pw + 'sol_census_all_2009_v2.csv')
        pop = pd.read_csv(pw + 'solo_pop_proj_2009.csv')
        aggs = df.loc[:, 'agg'].unique()
        geos = []
        for agg in aggs:
            geo = gpd.read_file(pw + 'sol_geo_' + agg.lower() + '.json')
            # Add an agg column, as the data and geometry need to be joined by id and agg
            geo.loc[:, 'agg'] = agg
            geos.append(geo)

        ret = cls.__transform(df, pop, geos)
        return cls(
            cen_df = ret[0],
            pop_df = ret[1],
            geos = ret[2],
        )
    
    @classmethod
    def load_pickle(cls,
                    folder:str = "/testData/", #file path of the folder to save in
                    aws:bool = True, # Whether to load from github or local
                    file_name:str = 'sol_geo.pickle' # file name of the saved class
                 ):
        '''
        A constuctor that initialises the object from aws pickle
        '''
        # Create a connection to AWS server
        client = s3_client()

        if aws:
            # Create the S3 object
            obj = client.get_object(
                Bucket = 'hobby-data',
                Key = file_name, 
            )

            # Read in the pickle
            try:
                tmp_geo = pickle.load(obj['Body'])
            except:
                raise ValueError("Issue dowloading pickle file from AWS.")
                
        else:
            # TODO work out how to make this a class method
            repo = Repo('.', search_parent_directories=True)
            pw = str(repo.working_tree_dir) + folder + file_name
            
            with open(pw, 'rb') as f:
                tmp_geo = pickle.load(f)
 
        
        return cls(
            cen_df = gpd.GeoDataFrame(tmp_geo['census']),
            pop_df = gpd.GeoDataFrame(tmp_geo['population']),
            geos = gpd.GeoDataFrame(tmp_geo['geo']),
        )
        
    
    @classmethod
    def gen_stored(cls,
                  json_sol:dict, # A geojson dataset
                 ): # A solmon geo class TODO work out how to return self here... (can't?)
        '''
        A constructor that creates a JSON serialised SolomonGeo object from a stored geopandas dataframe.
        The purpose of this is to allow the object to be stored JSON serialised in a DCC.Store object in 
        the browser before being deserialised and as an object.

        Note that storing and the reloading, will result in dropping the geometry.
        '''
        gdf = gpd.GeoDataFrame(json_sol['geojson'])
        cols = gdf.columns.str.extract(r'(.*): (.+)', expand=True)
        gdf.columns = pd.MultiIndex.from_arrays((cols[0], cols[1]))
        gdf.columns.names = [None]*2
        gdf.index.name = 'pk'
        return cls(
            geo_df = gdf
        )
    
    @classmethod
    def __transform(cls, 
                    df:pd.DataFrame, # The dataframe containing census data
                    pop_df:pd.DataFrame, # The dataset containing the population projection data
                    l_geos:[gpd.GeoDataFrame], # A list of geopandas dataframes containing 
                                                # the geographies 
                 ) -> gpd.GeoDataFrame: # Returns combined dataset
        '''
        Extract and return input datasets from file. Assumes correct format of input dataset, then
        Transform given raw input dataset into a cleaned and combined geopandas dataframe
        '''
        # TODO seperate out the geometry from the data.
        # TODO - make a function that tests that the geo and datasets both join

        geos = gpd.GeoDataFrame()
        for geo in l_geos:
            # Before combining, need to rename like columns
            # Rename columns and keep only necessary ones, Note that id can be province id, contsituency id etc.
            geo.columns = geo.columns.str.replace(r'^[a-zA-Z]+name$', 'geo_name', case = False, regex = True)
            # TODO this assumes the id key column is the first one (which so far it is...)
            geo.rename(columns = {geo.columns[0]:'id'}, inplace=True)

            geo = geo.loc[:, ['id', 'agg', 'geometry']] 

            # simplify the geography, use topo to preserver the topology between shapes
            topo = tp.Topology(geo, prequantize=False)
            geo = topo.toposimplify(720/43200).to_gdf() # old 360/43200

            geos = pd.concat([geos, geo])
            
        # Clean the geospatial dataframe
        geos.loc[:, 'year'] = '2009'
        
        # Clean the census data
        df = df.dropna()
        # Rename columns to be consistent across geography
        df.columns = df.columns.str.replace(r'^[a-zA-Z]+_name$', 'geo_name', case = False, regex = True)
        # id needs to change types twice so that it is a string of an int
        df = df.astype({'id': 'int'})#, 'male_pop':'int', 	'female_pop':'int', 'total_pop':'int'})
        df = df.astype({'id': 'str'})

        pop_df = pop_df.astype({'core: id': 'int'})
        pop_df = pop_df.astype({'core: id': 'str'})

        # Index is unique by type and geoname
        df['pk'] = df['geo_name'] + "_" + df["type"] 
        df = df.set_index("pk") 

        # Rename some of the census data
        df = df.rename(columns = {
                                'id':'core: id', 'agg':'core: agg', 'geo_name':'core: location',
                                'year':'core: year', 'type':'core: type'})

        # Test that the datasets all have geographies
        test_geo(df, geos)
        test_geo(pop_df, geos.loc[geos['agg'] == 'Province'])         

        # Convert into a multiindex dataframe, with hiearchical columns
        try:
            cols = df.columns.str.extract(r'(.*): (.+)', expand=True)
            df.columns = pd.MultiIndex.from_arrays((cols[0], cols[1]))
            df.columns.names = [None]*2

            cols2 = pop_df.columns.str.extract(r'(.*): (.+)', expand=True)
            pop_df.columns = pd.MultiIndex.from_arrays((cols2[0], cols2[1])) 
            pop_df.columns.names = [None]*2
        except:
            raise ValueError("Issue converting geopandas dataframe to multindex. \
                             Check that all columns have \': \' beside the following\
                             core columns: geometry, id, agg, year, type.")
                
        # return the transformed dataset
        return df, pop_df, geos


# SolomonGeo Data Class
This data class manipulates and stores the geospatial and regular data relating to each geography. It includes
the necessary data and methods to manipulate that data to build the resulting dash app.

In [ ]:
#| hide
show_doc(SolomonGeo)

---

### SolomonGeo

>      SolomonGeo (cen_df:pandas.core.frame.DataFrame,
>                  pop_df:pandas.core.frame.DataFrame,
>                  geos:geopandas.geodataframe.GeoDataFrame)

Load the solomon islands geography data 
Attributes:
    cen_df    Geopandas dataframe containing geographies and census data
    geo_levels    A list of the types of available aggregations
    census_vars    A dictionary of census variables in the dataset 
    data_type   Specifies whether the variable is a percentage or number
    locations A dictionary of locations accessed by the geography level

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| cen_df | DataFrame | A dataset containing the census data, |
| pop_df | DataFrame | A dataset contain the population projection data |
| geos | GeoDataFrame | A geodataframe containing geographies of data |

## Constructors for SolomonGeo class
These constudctors are used to build the SolomonGeo objects from various inputs

In [ ]:
#| hide
show_doc(SolomonGeo.read_test)

---

### SolomonGeo.read_test

>      SolomonGeo.read_test ()

Contsructor that initialises the object from files using the local testing data

In [ ]:
show_doc(SolomonGeo.load_pickle)

---

### SolomonGeo.load_pickle

>      SolomonGeo.load_pickle (folder:str='/testData/', aws:bool=True,
>                              file_name:str='sol_geo.pickle')

A constuctor that initialises the object from aws pickle

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| folder | str | /testData/ | file path of the folder to save in |
| aws | bool | True | Whether to load from github or local |
| file_name | str | sol_geo.pickle | file name of the saved class |

In [ ]:
#| hide
show_doc(SolomonGeo.gen_stored)

---

### SolomonGeo.gen_stored

>      SolomonGeo.gen_stored (json_sol:dict)

A constructor that creates a JSON serialised SolomonGeo object from a stored geopandas dataframe.
The purpose of this is to allow the object to be stored JSON serialised in a DCC.Store object in 
the browser before being deserialised and as an object.

Note that storing and the reloading, will result in dropping the geometry.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| json_sol | dict | A geojson dataset |

## Test that dataset and geography merge perfectly

In [ ]:
# TODO should this method be callable from the class
def test_geo(df:pd.DataFrame, # The dataframe to test
                geo:gpd.GeoDataFrame, # The geographys
                ):
    '''
    Tests that the provided geography and data merge perfectly. I.e. all data
    has a coresponding geography. The primary key to merge id and agg
    '''
    # Use outer join to see if anything is missing
    geo_df = geo.merge(df, left_on=['id', 'agg'], right_on=['core: id', 'core: agg'], how = 'outer', indicator = True)
    test_eq(sum(geo_df['_merge'] != 'both'), 0) # The total of non-joined rows should be 0



## Save SolomonGeo
Save as a pickled object

In [ ]:
#| export
@patch
def save_pickle(self:SolomonGeo,
                aws:bool = True, # Whether to save to aws or locally
                folder:str = "/testData/", #file path of the folder to save in, only necesasry for local saving
                file_name:str = 'sol_geo.pickle' # file name of the saved class
             ):
    '''
    Save a pickle of the SolomonGeo class in backblaze b2
    '''
    if aws:
      body_pickle = pickle.dumps(self.__dict__)
      try:
        client = s3_client()
        client.put_object(
            Bucket = 'hobby-data',
            Key = file_name, 
            Body = body_pickle
        )
      except:
         raise ValueError("Issue uploading pickle file to AWS.")
    else:
      repo = Repo('.', search_parent_directories=True)
      pw = str(repo.working_tree_dir) + folder + file_name
      
      f = open(pw, 'wb')
      pickle.dump(self.__dict__, f, 2)
      f.close()


In [ ]:
show_doc(SolomonGeo.save_pickle)

---

### SolomonGeo.save_pickle

>      SolomonGeo.save_pickle (aws:bool=True, folder:str='/testData/',
>                              file_name:str='sol_geo.pickle')

Save a pickle of the SolomonGeo class in backblaze b2

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| aws | bool | True | Whether to save to aws or locally |
| folder | str | /testData/ | file path of the folder to save in, only necesasry for local saving |
| file_name | str | sol_geo.pickle | file name of the saved class |

## Get Geo JSON
A getter method for the geometry portion of the dataset that returns a geoJson formated geography. 

It only includes the geography and location name as id

In [ ]:
#| export
@patch
def get_geojson(self:SolomonGeo, 
                geo_filter:str = None, # Filters the geojson to the requested aggregation 
               ) -> dict: # Geo JSON formatted dataset
    '''
    A getter method for the SolomonGeo class that returns a Geo JSON formatted dataset
    '''
    ret = self.geo_df
    # Only need geojson from one half of the dataset
    ret = ret.loc[ret['core']['type'] == self.type_default, :]
    ret = ret.set_index(ret.loc[:, ('core', 'location')]) # Change index to location for matching
    if geo_filter is not None:
        ret = ret.loc[ret['core']['agg'] == geo_filter, :]
    # Return only the core data to minimise the html size
    #return json.loads(ret.loc[:, ('core', 'geometry')].to_json())
    return json.loads(ret.loc[:, ('core', 'geometry')].to_json())

In [ ]:
#| hide
show_doc(SolomonGeo.get_geojson)

---

### SolomonGeo.get_geojson

>      SolomonGeo.get_geojson (geo_filter:str=None)

A getter method for the SolomonGeo class that returns a Geo JSON formatted dataset

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| geo_filter | str | None | Filters the geojson to the requested aggregation |
| **Returns** | **dict** |  | **Geo JSON formatted dataset** |

## Store JSON
A getter method that returns a dcc.Store object with the data of the `SolomonGeo` class
converted to a dictionary for storing with dash. We use a dictionary instead of geojson
as the time to convert dictionary back to geopandas is much faster than geojson. (0.01s vs 1.2s) This 
makes the map much more snappy and responsive

On storing, we drop the geometry as it won't store as json. Note that you can get around this
by using a geojson, but this increases reload time by a lot. 

In [ ]:
#| export
@patch
def get_store(self:SolomonGeo, 
            ) -> dcc.Store: # Geo JSON formatted dataset
    '''
    A getter method that returns a dcc.Store object with the data of the `SolomonGeo` class
    converted to a dictionary for storing with dash. 
    '''
    def hier_to_pandas(df:pd.DataFrame) -> pd.DataFrame:
        cols = df.columns.droplevel(1) + ": " + df.columns.droplevel(0)
        cols = cols.tolist()
        df.columns = cols
        return df

    cen_df = copy.copy(self.census)
    cen_df = hier_to_pandas(cen_df)

    pop_df = copy.copy(self.population)
    pop_df = hier_to_pandas(pop_df)
    
    geos = copy.copy(self.geo)    

    return dcc.Store(id="geo_df", data={"census": cen_df.to_dict(),
                                        "population": pop_df.to_dict(),
                                        "geo": geos.to_dict()})

In [ ]:
#| hide
show_doc(SolomonGeo.get_store)


---

### SolomonGeo.get_store

>      SolomonGeo.get_store ()

A getter method that returns a dcc.Store object with the data of the `SolomonGeo` class
converted to a dictionary for storing with dash.

## Get Dataframe
Returns a particular subset of the geo_df for further anaylsis

In [ ]:
#| export
@patch
def get_df(self:SolomonGeo, 
                geo_filter:str = None, # Filters the dataframe to the requested geography 
                var:str = None, # Selects an upper level 
                measure:str = None, # Selects the lower level variable, if var 1 is used, measure must be used.
                loc_filter:[str] = None, # Filters one of more locations
                # TODO remove hardcoding here?
                type_filter:str = 'Total', # Return either number of proportion
                agg_flag = False, # Whether to return the dataset aggregated for the given selection
                dataset = 'census', # Dataset, eiter census or 
               ) -> pd.DataFrame: # Pandas Dataframe containing population data
    '''
    A getter method for the SolomonGeo class that returns a pandas dataset containg
    the id variable and the total population variable. This is the minimal data required
    to display on the map. 
    '''
    if dataset == 'census': ret = self.census
    elif dataset == 'population': ret = self.population
    else: ValueError("get_df dataset must be: census or population")
    ret = ret.loc[ret['core']['type'] == type_filter, :] 
    ret = ret.set_index(ret.loc[:, ('core', 'location')]) # Change index to location as it's more desriptive
    # TODO check that filter is valid
    if geo_filter is not None:
        try:
            assert(geo_filter in ['Ward', 'Constituency', 'Province'])
        except:
            ValueError("Geo filter must be one of: ['Ward', 'Constituency', 'Province']")
        ret = ret.loc[ret['core']['agg'] == geo_filter, :]

    if loc_filter is not None:
        ret = ret.loc[ret['core']['location'].isin(loc_filter), :]

    # Return no core data to minimise the html size
    ret = ret.drop(columns = 'core', level=0)

    # Keep only selected column if required
    if measure is not None:
        try:
            assert(var is not None)
            assert(measure in self.census_vars[var])
        except:
            ValueError("If measure is set, var 1 must be set and the key value pair of var and measure must match")
        ret = ret[var].filter(items = [measure])
    elif var is not None:
        # Keep all values from upper level column
        ret = ret[var]
        
    return pd.DataFrame(ret)

In [ ]:
#| hide
show_doc(SolomonGeo.get_df)

---

### SolomonGeo.get_df

>      SolomonGeo.get_df (geo_filter:str=None, var:str=None, measure:str=None,
>                         loc_filter:[<class'str'>]=None,
>                         type_filter:str='Total', agg_flag=False,
>                         dataset='census')

A getter method for the SolomonGeo class that returns a pandas dataset containg
the id variable and the total population variable. This is the minimal data required
to display on the map.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| geo_filter | str | None | Filters the dataframe to the requested geography |
| var | str | None | Selects an upper level |
| measure | str | None | Selects the lower level variable, if var 1 is used, measure must be used. |
| loc_filter | [<class 'str'>] | None | Filters one of more locations |
| type_filter | str | Total | Return either number of proportion |
| agg_flag | bool | False | Whether to return the dataset aggregated for the given selection |
| dataset | str | census | Dataset, eiter census or |
| **Returns** | **DataFrame** |  | **Pandas Dataframe containing population data** |

## Aggregate Dataframe
Returns an aggregate pandas series 

In [ ]:
#| export
@patch
def agg_df(self:SolomonGeo, 
                geo_filter:str = None, # Filters the dataframe to the requested geography 
                var:str = None, # Selects an upper level 
                measure:str = None, # Selects the lower level variable, if var 1 is used, measure must be used.
                loc_filter:[str] = None, # Filters one of more locations
                # TODO remove hardcoding here?
                type_filter:str = 'Total', # Return either number of proportion
               ) -> pd.Series: # Pandas data series containing aggregated and filtered data
    '''
    A getter method for the SolomonGeo class that calls get_df to get a spcific and then further 
    aggregates that dataset so that the proportion is the weighted proportion
    '''
    df = self.get_df(geo_filter, var, measure, loc_filter)

    if type_filter == 'Total':
        df = df.sum()
    elif type_filter == 'Proportion':
        df = df.sum() / df.sum().sum() * 100
    else:
        raise ValueError('The type passed to the aggregate function must be one of the following: \'Total\', \'Proportion\'.')
    return df

In [ ]:
show_doc(agg_df)

---

[source](https://github.com/fastai/nbdev/blob/master/nbdev/showdoc.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### show_doc

>      show_doc (sym, renderer=None, name:str|None=None, title_level:int=3)

Show signature and docstring for `sym`

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| sym |  |  | Symbol to document |
| renderer | NoneType | None | Optional renderer (defaults to markdown) |
| name | str \| None | None | Optionally override displayed name of `sym` |
| title_level | int | 3 | Heading level to use for symbol name |

# Testing

In [ ]:
sol_geo = SolomonGeo.read_test()

In [ ]:
sol_geo.population

core                                          Age                    \
       id  loc_name       agg   type  year Age_Bracket Numerical_Bracket   
0       1  Choiseul  Province  Total  2010         0-4                 0   
1       1  Choiseul  Province  Total  2010         5-9                 5   
2       1  Choiseul  Province  Total  2010       10-14                10   
3       1  Choiseul  Province  Total  2010       15-19                15   
4       1  Choiseul  Province  Total  2010       20-24                20   
...   ...       ...       ...    ...   ...         ...               ...   
2715   10   Honiara  Province  Total  2025       60-64                60   
2716   10   Honiara  Province  Total  2025       65-69                65   
2717   10   Honiara  Province  Total  2025       70-74                70   
2718   10   Honiara  Province  Total  2025       75-79                75   
2719   10   Honiara  Province  Total  2025         80+                80   

     Population                 
          Males Females  Total  
0         2,279   2,060  4,339  
1         2,131   2,002  4,133  
2         1,778   1,640  3,418  
3         1,379   1,342  2,721  
4         1,192   1,083  2,275  
...         ...     ...    ...  
2715      1,406   1,236  2,642  
2716        955     791  1,746  
2717        619     466  1,085  
2718        318     275    593  
2719        202     172    374  

[2720 rows x 10 columns]

In [ ]:
sol_geo.locations

{'Province': array(['Choiseul', 'Honiara', 'Western', 'Isabel', 'Central',
        'Rennell-Bell', 'Guadalcanal', 'Malaita', 'Makira-Ulawa', 'Temotu'],
       dtype=object),
 'Constituency': array(['South Choiseul', 'North West Choiseul', 'East Choiseul',
        'East Honiara', 'Central Honiara', 'West Honiara',
        'Shortland Islands', 'North Vella la Vella',
        'South Vella la Vella', 'Ranongga-Simbo', 'Gizo-Kolombangara',
        'West New Georgia-Vonavona', 'North New Georgia',
        'South New Georgia-Rendova', 'Marovo', 'Hograno-Kia-Havulei',
        'Marigne-Kokota', 'Gao-Bugotu', 'Nggela', 'Savo-Russull Islands',
        'Rennell-Bellona', 'North West Guadalcanl', 'West Guadalcanal',
        'South Guadalcanal', 'East Guadalcanal', 'East-Central',
        'North-East', 'North Guadalcanal', 'Central Guadalcanal',
        'North Malaita', 'Lau-Mbaelela', 'Baegu-Asifola', 'Fataleka',
        "West Kwara'ae", "Central Kwara'ae", 'East Malaita',
        'Auki-Langalanga'

## Save and Load

Test that the newly created solomon geo object can be saved to aws

In [ ]:
sol_geo.save_pickle()

Test that we can connect to the aws s3 bucket

In [ ]:
# Create the S3 object
obj = s3_client().get_object(
    Bucket = 'hobby-data',
    Key = 'test.txt', 
)

# Read in the pickle
try:
    data = obj['Body'].read()
except:
    raise ValueError("Issue dowloading test file from AWS.")

In [ ]:
#| slow
SolomonGeo.load_pickle('/testData/', aws=True)

<__main__.SolomonGeo>

Test that gen_stored creates a copy correctly from a json serialised dataframe

In [ ]:
stored_geo = sol_geo.get_store()
# TODO I need to potentially created a function to check if two objects are the same

KeyError: "['core: geometry'] not found in axis"

In [ ]:
restored_geo = SolomonGeo.gen_stored(stored_geo.data)


In [ ]:
restored_geo.get_df(geo_filter = 'Constituency', loc_filter=['Nggela', 'East AreAre'])

Key Statistics Main source of household drinking water  \
                 Total Households             metered SIWA drinking water   
(core, location)                                                            
East AreAre                  1105                                     1.0   
Nggela                       3315                                    26.0   

                                                                            \
                 communal standpipe private water tank communal water tank   
(core, location)                                                             
East AreAre                   815.0               12.0                 9.0   
Nggela                       1557.0              258.0               437.0   

                                                                      \
                 well - protected well - unprotected river or stream   
(core, location)                                                       
East AreAre                   1.0                3.0           223.0   
Nggela                       52.0               13.0           868.0   

                                                               ...  \
                 bottled water other source of drinking water  ...   
(core, location)                                               ...   
East AreAre                0.0                           41.0  ...   
Nggela                     1.0                          103.0  ...   

                 Household money received from remittances  \
                                  1000 - 1499 S.I. dollars   
(core, location)                                             
East AreAre                                           41.0   
Nggela                                                25.0   

                                              \
                 more than 1500 S.I. dollars   
(core, location)                               
East AreAre                             15.0   
Nggela                                  59.0   

                 Main source of household income in last 12 months  \
                                                         No income   
(core, location)                                                     
East AreAre                                                   37.0   
Nggela                                                        49.0   

                                                                            \
                 Wages Salary Own business Sale fish crop craft Land lease   
(core, location)                                                             
East AreAre              89.0        101.0                650.0        0.0   
Nggela                  558.0        327.0               1878.0        0.0   

                                                      
                 House rent Remittances Other source  
(core, location)                                      
East AreAre             2.0        31.0        195.0  
Nggela                  4.0        75.0        424.0  

[2 rows x 66 columns]

In [ ]:
# TODO create a test to check that two object are the same. Probably need to write defualt behaviour into
# object. Maybe use fastcore test_eq
#test_eq(stored_geo.geo_df, sol_geo.geo_df)

Test filtering of down to multiple locations

In [ ]:
#test_eq(sol_geo.get_df(geo_filter = 'constituency')['total_pop'].sum(), const_df['total_pop'].sum())
sol_geo.get_df(geo_filter = 'Constituency', loc_filter=['Nggela', 'East AreAre'])

Key Statistics Main source of household drinking water  \
                 Total Households             metered SIWA drinking water   
(core, location)                                                            
East AreAre                  1105                                     1.0   
Nggela                       3315                                    26.0   

                                                                            \
                 communal standpipe private water tank communal water tank   
(core, location)                                                             
East AreAre                   815.0               12.0                 9.0   
Nggela                       1557.0              258.0               437.0   

                                                                      \
                 well - protected well - unprotected river or stream   
(core, location)                                                       
East AreAre                   1.0                3.0           223.0   
Nggela                       52.0               13.0           868.0   

                                                               ...  \
                 bottled water other source of drinking water  ...   
(core, location)                                               ...   
East AreAre                0.0                           41.0  ...   
Nggela                     1.0                          103.0  ...   

                 Household money received from remittances  \
                                  1000 - 1499 S.I. dollars   
(core, location)                                             
East AreAre                                           41.0   
Nggela                                                25.0   

                                              \
                 more than 1500 S.I. dollars   
(core, location)                               
East AreAre                             15.0   
Nggela                                  59.0   

                 Main source of household income in last 12 months  \
                                                         No income   
(core, location)                                                     
East AreAre                                                   37.0   
Nggela                                                        49.0   

                                                                            \
                 Wages Salary Own business Sale fish crop craft Land lease   
(core, location)                                                             
East AreAre              89.0        101.0                650.0        0.0   
Nggela                  558.0        327.0               1878.0        0.0   

                                                      
                 House rent Remittances Other source  
(core, location)                                      
East AreAre             2.0        31.0        195.0  
Nggela                  4.0        75.0        424.0  

[2 rows x 66 columns]

In [ ]:
# TODO work out how to check save and load are the same as a test
# Might need to wreite and equality function

In [ ]:
# TODO should do a count check for the geojson similar to this
sol_geo.geo_df[sol_geo.geo_df['core']['agg'] == 'Constituency'].count()

core                                               geometry                100
                                                   id                      100
                                                   agg                     100
                                                   year                    100
                                                   location                100
                                                                          ... 
Main source of household income in last 12 months  Sale fish crop craft    100
                                                   Land lease              100
                                                   House rent              100
                                                   Remittances             100
                                                   Other source            100
Length: 72, dtype: int64

# TODO check that all proportions are less than 0

In [ ]:
# TODO filter and test equality?
test = sol_geo.get_geojson(geo_filter = 'Ward')

The sum after merging and filtering should equal the sum from the raw dataset.

In [ ]:
test_eq(sol_geo.agg_df(geo_filter = 'Constituency',var = 'Key Statistics', measure = 'Total Households')[0], 
        const_df.loc[const_df['type'] == 'Total'].loc[const_df['agg'] == 'Constituency']['Key Statistics: Total Households'].sum())

/tmp/ipykernel_11743/1266885615.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test_eq(sol_geo.agg_df(geo_filter = 'Constituency',var = 'Key Statistics', measure = 'Total Households')[0],


In [ ]:
sol_geo.agg_df(geo_filter = 'Constituency', var='Key Statistics', measure = 'Total Households')

Total Households    91251
dtype: int64

In [ ]:
sol_geo.get_df(geo_filter = 'Constituency', var='Key Statistics', measure = 'Total Households', loc_filter=['West Guadalcanal']).values[0]

array([1671])

In [ ]:
sol_geo.geo_df['core'].filter(items = ['id'])

,id
pk,
Choiseul_Total,1
Choiseul_Proportion,1
Honiara_Total,10
Honiara_Proportion,10
Western_Total,2
...,...
Vanikoro_Proportion,915
Tikopia_Total,916
Tikopia_Proportion,916


In [ ]:
# TODO -test that they all sum to one
test_agg = sol_geo.agg_df(geo_filter = 'Constituency', type_filter='Proportion', var = "Main source of household drinking water")
test_eq(test_agg.sum(), 100.0)
test_agg

metered SIWA drinking water        9.168119
communal standpipe                35.114136
private water tank                12.471096
communal water tank               10.619062
well - protected                   2.081073
well - unprotected                 1.781898
river or stream                   24.543293
bottled water                      0.611500
other source of drinking water     3.609823
dtype: float64

In [ ]:
test_agg.sum()

100.0

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()